# Voltage Regulator Waveform Demo

This notebook demonstrates publishing and reading back parametric waveform data with the NI Measurement Data Store. To demonstrate this, it uses simulated data of the dynamic response of a voltage regulator circuit. The simulated voltage regulator is set to regulate to 5V. The input voltage is the measurement parameter and varies from 4.5V to 12V. The data shows how the regulator can only regulate voltages within a certain range. If the input voltage is too large, it will not be able to pin the output voltage to its intended 5V.

## Set up the test result for publishing data

First, we're going to set up our test result metadata and get everything ready to publish

In [57]:
from ni.datastore.client import Client
from ni.measurements.data.v1.data_store_pb2 import TestResult
from ni.measurements.metadata.v1.metadata_store_pb2 import Operator, TestStation

client = Client()

operator = Operator(operator_name="James Bowery", role="Test Architect")
operator_id = client.create_operator(operator)

test_station = TestStation(test_station_name="TestStation_12")
test_station_id = client.create_test_station(test_station)

test_result = TestResult(
    test_result_name="voltage regulator waveforms",
    operator_id=operator_id,
    test_station_id=test_station_id,
)
test_result_id = client.create_test_result(test_result)
print(f"created test result id: {test_result_id}")


created test result id: 8d4aedf9-dfb2-4054-8d5a-f9a8a6dc95c8


## Simulate the data

Next, we'll simulate the voltage regulator data.

In [58]:
import numpy as np
import plotly.graph_objects as go

def simulate_voltage_regulator(input_voltage, time_ms, target_voltage=5.0):
    """
    Simulates the output voltage of a voltage regulator over time.
    """
    output = np.zeros_like(time_ms, dtype=float)
    startup_delay = 50  # ms
    ramp_duration = 200  # ms
    overshoot = 0.2 if input_voltage >= target_voltage else 0.0
    max_output = min(input_voltage, target_voltage + overshoot)

    for i, t in enumerate(time_ms):
        if t < startup_delay:
            output[i] = 0
        elif t < startup_delay + ramp_duration:
            ramp_progress = (t - startup_delay) / ramp_duration
            output[i] = ramp_progress * max_output
        else:
            # Simulate regulation behavior
            if input_voltage < target_voltage:
                output[i] = input_voltage
            elif input_voltage <= 8:
                output[i] = target_voltage + overshoot * np.exp(-(t - startup_delay - ramp_duration)/300)
            elif input_voltage <= 10:
                output[i] = target_voltage + 0.5 + 0.2 * np.sin(0.01 * t)
            else:
                output[i] = target_voltage + 1.0 + 0.5 * np.sin(0.01 * t)
    return output

# Time in milliseconds
time_ms = np.linspace(0, 1000, 500)

# Input voltages to simulate
input_voltages = [4.5, 7.0, 9.0, 12.0]
colors = ['blue', 'green', 'orange', 'red']

# Simulate each input voltage
vout = []
for vin, color in zip(input_voltages, colors):
    vout.append(simulate_voltage_regulator(vin, time_ms))


## Publish the Simulated Data

Publish the simulated `AnalogWaveform`s (as measurements) along with the input voltage parameter values (as conditions)

In [59]:
from datetime import datetime, timedelta, timezone
from nitypes.scalar import Scalar
from nitypes.waveform import AnalogWaveform
from nitypes.waveform import Timing
import numpy as np
from ni.measurements.data.v1.data_store_pb2 import Step

step = Step(step_name="Initial step", test_result_id=test_result_id)
step_id = client.create_step(step)
print(f"created step id: {step_id}")
published_measurement_ids = []
i = 0
for vout_for_input_voltage in vout:
    waveform = AnalogWaveform(
        sample_count=len(vout_for_input_voltage),
        raw_data=np.array(vout_for_input_voltage),
        timing=Timing.create_with_regular_interval(
            timedelta(seconds=1e-3),
            datetime.now(timezone.utc)
        )
    )
    published_measurement = client.publish_measurement(
        measurement_name="voltage_response",
        value=waveform,
        step_id=step_id,
    )
    published_measurement_ids.append(published_measurement.published_measurement_id)
    print(f"The published measurement id is {published_measurement.published_measurement_id}")
    scalar = Scalar(value=input_voltages[i], units="V")
    published_condition = client.publish_condition(
        "input_voltage",
        "integer",
        scalar,
        step_id
    )
    i += 1

created step id: c8a13e5b-d377-4b90-b923-019173e7d012
The published measurement id is 096efb90-b837-4c50-ab5f-8a1cfbf03ba7
The published measurement id is 9c6c522a-ced4-41aa-8985-c2363322a1af
The published measurement id is 5002d388-7366-4680-987f-9dd77f4f3ae4
The published measurement id is 02581041-3ee1-4bcb-9391-69811fcdb04c


## Read Published Data

Read back the published data and plot them using the parameter (condition) values and the timing data from the stored `AnalogWaveform`s and display them in a plotly graph.

In [60]:
from nitypes.vector import Vector

# Create Plotly figure
fig = go.Figure()

conditions = client.query_conditions(odata_query=f"$filter=stepid eq {step_id}")
condition = next(iter(conditions), None)
condition_values = []
if condition is not None:
    condition_data = client.read_data(condition.moniker, expected_type=Vector)
    units = condition_data.units
    for val in condition_data:
        condition_values.append(str(val) + units)
measurements = client.query_measurements(odata_query=f"$filter=stepid eq {step_id}")
sorted_measurements = sorted(measurements, key=lambda m: m.parametric_index)
for measurement in sorted_measurements:
    waveform = client.read_data(measurement.moniker, expected_type=AnalogWaveform)
    color = colors[measurement.parametric_index]
    timing = waveform.timing
    fig.add_trace(go.Scatter(
        x=np.arange(len(waveform.raw_data)) * waveform.timing.sample_interval.total_seconds() * 1000,
        y=waveform.raw_data,
        mode='lines',
        name=f'Input {condition_values[measurement.parametric_index]}',
        line=dict(color=color)
    ))


# Update layout
fig.update_layout(
    title='Voltage Regulator Output Simulation',
    xaxis_title='Time (ms)',
    yaxis_title='Output Voltage (V)',
    legend_title='Input Voltage',
    template='plotly_white'
)

fig.show()